In [5]:
# Manifold Markets tool for user loan statistics

In [6]:
###############################
# Get user loan info from API #
###############################

####### CONFIG ##########
# Manifold username
username = 'parhizj'
# Manifold Market API key
api_key = "YOUR-API-KEY-HERE"
#########################

import requests
import time
import os
import json
from datetime import datetime

bets_url_by_username = 'https://manifold.markets/api/v0/bets?username='

json_filepath = 'manifold-loans-bets-all.json'

# API request headers
headers = {
    "Authorization": f"Key {api_key}",
}

In [7]:
# 1 second delay between requests
def get_username_bets_before(user_name, before=None):
    bets_before = []
    # for first run
    orig_url = bets_url_by_username + user_name
    url = orig_url
    total_count = 0
    if before is not None:
        url += '&before=' + before
    while url:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            num_bets = len(data)
            if isinstance(data, list) and num_bets > 0:
                last_bet_id = None
                for bet in data:
                    if isinstance(bet, dict) and "id" in bet:
                        before = bet["id"]
                        total_count += 1
                        bets_before.append(bet)
                time.sleep(1)  # Delay between requests to avoid hammering
                # assume the last bet is the oldest one in the JSON
                url = orig_url + '&before=' + before
            else:
                if isinstance(data, dict) and 'error' in data:
                    print("Error, data from API not a list or no data: {data}")
                url = None
            if num_bets < 1000:
                # requested a 1000, so if there is less we have hit the end of the bets
                url = None
        else:
            url = None
    return bets_before


In [8]:
bets = get_username_bets_before(username)

In [9]:
if bets is not None:
    with open(json_filepath, 'w') as f:
        json.dump(bets, f)

In [10]:
with open(json_filepath, 'r') as f:
    all_bets = json.load(f)

bets_by_contract = {}
for bet in all_bets:
    if 'loanAmount' in bet and 'contractId' in bet:
        contractId = bet['contractId']
        contractBets = []
        if contractId in bets_by_contract:
            contractBets = bets_by_contract[contractId]
        contractBets.append(bet)
        bets_by_contract[contractId] = contractBets
totalLoans = 0
contractNetLoans = {}
for contract_id, bets in bets_by_contract.items():
    contractLoan = 0
    for bet in bets:
        if 'loanAmount' in bet:
            contractLoan += bet['loanAmount']
    # print(f"Contract net loan for {contract_id}: {contractLoan:1.0f}")
    contractNetLoans[contract_id] = contractLoan
    totalLoans += contractLoan
    
sorted_contractNetLoans = dict(sorted(contractNetLoans.items(), key=lambda x:x[1], reverse=True))
for contract_id, loanAmount in sorted_contractNetLoans.items():
    if loanAmount >= 0.5:
        print(f"Contract net loan for {contract_id}: {loanAmount:1.0f}")

Contract net loan for w8Axni4eMBky1Mg0IxIL: 105
Contract net loan for DgT3wPneFl9GPjC9MUFw: 95
Contract net loan for TnmU6MpafpeJViQOFaNW: 90
Contract net loan for cpGe9ErhdnZ5yhjzGEpO: 89
Contract net loan for SZQH87RNsqa1kplkPn5F: 75
Contract net loan for sG9xUB5jxzmUHZWkSW0d: 41
Contract net loan for Z95HfNHwqaD6EkChmIDJ: 41
Contract net loan for fyZff32mPbXBxd5IfWeq: 40
Contract net loan for NNsQvVXnJwhYaSWNrIT0: 39
Contract net loan for qp7qv5PeIY0ELcMo4cmQ: 35
Contract net loan for ePuLMhmIC1XSHIMjDJeN: 33
Contract net loan for 6F5ZEC3VpY9WY6C3Ay7i: 32
Contract net loan for BKx2ha4LBj5uQpD4bdO2: 31
Contract net loan for Bjz06YJmRycbfbkE8anm: 30
Contract net loan for avgjyL9FAWEmeaGEbFyM: 28
Contract net loan for ieIbqFDlQlnKh7PKAQT1: 28
Contract net loan for onMAJ1OtuHSRdIXwxW2p: 25
Contract net loan for dib8XAR3WIdrAz9RgFnn: 25
Contract net loan for SVk1ZnhV2A7ZL3CEn3SB: 24
Contract net loan for 6zqhpoEPAWXalDUGwsPp: 24
Contract net loan for nNYuLGSNC3cE443aabHa: 23
Contract net

In [13]:
print(f"# Bets: {len(all_bets)}")
print(f"Contracts: {len(bets_by_contract)}")
print(f"Net loan amount: {totalLoans:1.0f}")

# Bets: 7774
Contracts: 374
Net loan amount: 2129
